In [3]:
import numpy as np
import scipy.linalg as lin
from scipy.sparse import diags
import random
from scipy.sparse.linalg import spsolve
from sys import getsizeof as gso
import time

# Задача 1

In [1]:
def rnd_band(n, m):
    A=np.random.random((n, m))
    r=np.random.randint(max(n, m))
    A=np.tril(A, r)
    A=np.triu(A, -r)
    return A

def const_band(n):
        r=np.random.randint(1, n)
        x=[i for i in range(-r, r+1)]
        y=np.random.random(len(x))
        return diags(y, x, shape=(n, n)).toarray()

def make_disturb(A, eps):
    return A+np.random.uniform(low=0, high=eps, size=np.shape(A))

#далее предполагается, что массив передаётся на вход в виде, как в примере
def gen_by_vectors(v_array):
    N=len(v_array)
    n=0
    for i in range(N):
        if len(v_array[i])>n:
            n=len(v_array[i])
    result=np.zeros((n, n))
    for i in range(N):
        if i<=N/2:
            result+=np.diag(v_array[i], -(n-len(v_array[i])))
        else:
            result+=np.diag(v_array[i], n-len(v_array[i]))
    return result

def show_exmp():
    y=[[42], [1, 2], [1, 2, 3, 4], [1, 1]]
    print(gen_by_vectors(y))
    return

show_exmp()

[[ 1.  0.  1.  0.]
 [ 0.  2.  0.  1.]
 [ 1.  0.  3.  0.]
 [42.  2.  0.  4.]]


In [5]:
def gen_band_1(N, m):
    diagonals=np.ones((5, N))
    return diags(diagonals, [0, -1, 1, -m, m]).toarray()

def gen_and_solve_1(N, m):
    A=gen_band_1(N, m)
    b=np.random.random((N, 1))
    time_1=time.time_ns()
    x=lin.solve(A, b)
    time_2=time.time_ns()
    return time_2-time_1, gso(A), x

def gen_band_2(N, m):
    diagonals=np.ones((5, N))
    A = np.zeros((2 * m + 1, N))
    A[0], A[m - 1], A[m], A[m + 1], A[2 * m]=diagonals[0], diagonals[1], diagonals[2], diagonals[3], diagonals[4]
    return A

def gen_and_solve_2(N, m):
    A=gen_band_2(N, m)
    b=np.random.random((N, 1))
    time_1=time.time_ns()
    x=lin.solve_banded((m, m), A, b)
    time_2=time.time_ns()
    return time_2-time_1, gso(A), x


def gen_band_3(N, m):
    diagonals=np.ones((5, N))
    return diags(diagonals, [0, -1, 1, -m, m])

def gen_and_solve_3(N, m):
    A=gen_band_3(N, m)
    b=np.random.random((N, 1))
    time_1=time.time_ns()
    x = spsolve(A, b)
    time_2=time.time_ns()
    return time_2-time_1, gso(A), x

In [6]:
#сравним между собой методы по времени работы
print('Метод 1: {}, Метод 2: {}, Метод 3: {}'.format(gen_and_solve_1(10, 3)[1], gen_and_solve_2(10, 3)[1], gen_and_solve_3(10, 3)[1]))


Метод 1: 912, Метод 2: 672, Метод 3: 48


# Задача 5



In [7]:
import numpy as np
import scipy.linalg as lin
import copy

def FPI(A, b, error=1e-9):
    E=np.eye(np.shape(A)[0])
    tau=1/(max(lin.eig(A)[0])+min(lin.eig(A)[0]))
    R=E-tau*A
    b=tau*b
    x=np.ones((np.shape(A)[0], 1))
    x=R@x+b
    I=10000
    for i in range(I):
        x=R@x+b
        r=R@x+b-x
        if r.T@r<error**2:
            return x
    print('FPI diverged')
    return -1

def Ga_m(A, b, error=1e-9):
    n=np.shape(A)[0]
    x=np.ones((n, 1))
    y=copy.deepcopy(x)
    r=A@x-b
    norm_2=lin.norm(r, 2)
    K=10000
    for k in range(K):
        e=lin.solve(A, -r)
        x+=e
        if lin.norm(r, 2)<error:
            return x
        r=A@x-b
    print("Method failed to converge")
    return -1

def compare(N):
    for i in range(2, N+1):
        A=np.random.randint(5, size=(i, i))-2.5
        A=A.T@A+np.eye(i)
        b=np.random.randint(5, size=(i, 1))-2.5
        print('FPI method for {} gives \n{}.'.format(i, FPI(A, b)), 'Ga_m method for {} gives \n{}.'.format(i, Ga_m(A, b)), sep='\n')



In [8]:
N=50
compare(N)

FPI method for 2 gives 
[[1.42857149e-01+0.j]
 [2.76788569e-09+0.j]].
Ga_m method for 2 gives 
[[0.14285714]
 [0.        ]].
FPI method for 3 gives 
[[ 0.26829269+0.j]
 [ 0.18292683+0.j]
 [-0.35365853+0.j]].
Ga_m method for 3 gives 
[[ 0.26829268]
 [ 0.18292683]
 [-0.35365854]].
FPI method for 4 gives 
[[ 1.06373817+0.j]
 [-1.28596037+0.j]
 [-0.99138674+0.j]
 [ 0.71231697+0.j]].
Ga_m method for 4 gives 
[[ 1.06373816]
 [-1.28596038]
 [-0.99138674]
 [ 0.71231697]].
FPI method for 5 gives 
[[-1.30125328+0.j]
 [ 1.34867991+0.j]
 [ 0.00833818+0.j]
 [-0.56316131+0.j]
 [ 0.61515939+0.j]].
Ga_m method for 5 gives 
[[-1.3012533 ]
 [ 1.34867993]
 [ 0.00833818]
 [-0.56316132]
 [ 0.6151594 ]].
FPI method for 6 gives 
[[ 0.37047582+0.j]
 [-1.88442046+0.j]
 [ 1.35410513+0.j]
 [ 0.63689084+0.j]
 [-0.06906473+0.j]
 [-0.13842193+0.j]].
Ga_m method for 6 gives 
[[ 0.37047582]
 [-1.8844205 ]
 [ 1.35410516]
 [ 0.63689085]
 [-0.06906473]
 [-0.13842193]].
FPI method for 7 gives 
[[-0.7834899 +0.j]
 [ 0.164

FPI method for 22 gives 
[[-0.84533416+0.j]
 [-0.9390782 +0.j]
 [ 0.67695065+0.j]
 [-0.07733976+0.j]
 [ 0.56800003+0.j]
 [-0.21130975+0.j]
 [ 0.38041432+0.j]
 [ 0.45236151+0.j]
 [-0.53207198+0.j]
 [-0.49791237+0.j]
 [ 0.00813974+0.j]
 [-0.07298585+0.j]
 [-0.10851122+0.j]
 [ 0.08650451+0.j]
 [-0.48223805+0.j]
 [ 0.40393803+0.j]
 [-0.2802627 +0.j]
 [ 0.7118418 +0.j]
 [ 0.02458042+0.j]
 [ 0.60881996+0.j]
 [ 0.60134384+0.j]
 [ 0.12867005+0.j]].
Ga_m method for 22 gives 
[[-0.84533423]
 [-0.93907819]
 [ 0.67695076]
 [-0.07733978]
 [ 0.56800003]
 [-0.21130979]
 [ 0.38041429]
 [ 0.45236156]
 [-0.53207198]
 [-0.49791237]
 [ 0.00813978]
 [-0.07298591]
 [-0.10851126]
 [ 0.08650454]
 [-0.48223805]
 [ 0.40393805]
 [-0.28026274]
 [ 0.71184186]
 [ 0.02458041]
 [ 0.60882   ]
 [ 0.60134389]
 [ 0.12867003]].
FPI method for 23 gives 
[[-0.18347934+0.j]
 [ 0.57643222+0.j]
 [-0.22202596+0.j]
 [ 0.14173639+0.j]
 [ 0.05235841+0.j]
 [-0.27133658+0.j]
 [-0.60116603+0.j]
 [ 0.80672046+0.j]
 [ 0.04019427+0.j]
 

FPI method for 31 gives 
[[ 0.13992675+0.j]
 [-0.63386651+0.j]
 [-0.08794467+0.j]
 [ 1.15015552+0.j]
 [ 0.27023739+0.j]
 [-0.68196131+0.j]
 [-0.55018128+0.j]
 [-1.03423156+0.j]
 [ 1.03791811+0.j]
 [-0.6814483 +0.j]
 [-0.23913742+0.j]
 [ 0.19582808+0.j]
 [-0.40716839+0.j]
 [ 0.85633555+0.j]
 [-0.14291623+0.j]
 [ 0.1344996 +0.j]
 [-0.20327055+0.j]
 [-0.53032944+0.j]
 [-0.11629619+0.j]
 [ 0.60948971+0.j]
 [ 0.82447901+0.j]
 [ 0.28897481+0.j]
 [-0.09322323+0.j]
 [ 0.60310906+0.j]
 [ 1.33523863+0.j]
 [-0.21437538+0.j]
 [ 0.63141432+0.j]
 [-0.38675669+0.j]
 [ 0.11058191+0.j]
 [-0.82890113+0.j]
 [-0.2450562 +0.j]].
Ga_m method for 31 gives 
[[ 0.13992681]
 [-0.63386657]
 [-0.08794467]
 [ 1.15015564]
 [ 0.27023746]
 [-0.68196141]
 [-0.55018128]
 [-1.03423165]
 [ 1.03791825]
 [-0.6814483 ]
 [-0.23913746]
 [ 0.1958281 ]
 [-0.40716836]
 [ 0.85633559]
 [-0.14291635]
 [ 0.1344996 ]
 [-0.20327058]
 [-0.53032949]
 [-0.11629621]
 [ 0.60948979]
 [ 0.82447914]
 [ 0.28897482]
 [-0.09322323]
 [ 0.60310916

FPI method for 38 gives 
[[-0.05459783+0.j]
 [-0.38539462+0.j]
 [-0.13214498+0.j]
 [ 0.25961052+0.j]
 [ 0.11315105+0.j]
 [-0.06349223+0.j]
 [-0.28569446+0.j]
 [-0.4182527 +0.j]
 [ 0.0775167 +0.j]
 [-0.17998595+0.j]
 [-0.1229056 +0.j]
 [ 0.08724814+0.j]
 [ 0.03849205+0.j]
 [ 0.36069932+0.j]
 [ 0.4272887 +0.j]
 [-0.24623766+0.j]
 [-0.30977445+0.j]
 [ 0.14710333+0.j]
 [-0.02802255+0.j]
 [ 0.326142  +0.j]
 [ 0.40719529+0.j]
 [ 0.27317761+0.j]
 [ 0.27213525+0.j]
 [ 0.04358332+0.j]
 [ 0.15176503+0.j]
 [ 0.18162631+0.j]
 [-0.0262387 +0.j]
 [-0.01234205+0.j]
 [-0.48983297+0.j]
 [-0.07510103+0.j]
 [-0.14671204+0.j]
 [ 0.02256382+0.j]
 [-0.13511192+0.j]
 [ 0.07998373+0.j]
 [ 0.2497727 +0.j]
 [-0.03885599+0.j]
 [-0.09266032+0.j]
 [-0.3965525 +0.j]].
Ga_m method for 38 gives 
[[-0.05459798]
 [-0.38539481]
 [-0.13214507]
 [ 0.25961066]
 [ 0.11315105]
 [-0.06349224]
 [-0.28569454]
 [-0.4182528 ]
 [ 0.07751668]
 [-0.1799859 ]
 [-0.12290559]
 [ 0.08724816]
 [ 0.03849186]
 [ 0.36069948]
 [ 0.42728884]


FPI method for 45 gives 
[[ 0.09111644+0.j]
 [-0.12859608+0.j]
 [ 0.00287101+0.j]
 [-0.1848015 +0.j]
 [ 0.25221396+0.j]
 [-0.09474535+0.j]
 [ 0.3079863 +0.j]
 [-0.34446526+0.j]
 [-0.1829817 +0.j]
 [-0.06796645+0.j]
 [-0.00168067+0.j]
 [-0.08683345+0.j]
 [ 0.25383732+0.j]
 [ 0.08415627+0.j]
 [ 0.1182583 +0.j]
 [ 0.18092008+0.j]
 [ 0.18125946+0.j]
 [-0.26642597+0.j]
 [-0.22547688+0.j]
 [ 0.08895399+0.j]
 [ 0.31995254+0.j]
 [ 0.42993243+0.j]
 [-0.03723835+0.j]
 [-0.01460924+0.j]
 [-0.01145208+0.j]
 [ 0.26841763+0.j]
 [ 0.04691302+0.j]
 [-0.11929473+0.j]
 [-0.205502  +0.j]
 [-0.25774404+0.j]
 [-0.33843664+0.j]
 [-0.02703289+0.j]
 [-0.2514564 +0.j]
 [-0.19207061+0.j]
 [-0.01208693+0.j]
 [-0.21596682+0.j]
 [ 0.10927541+0.j]
 [ 0.11481102+0.j]
 [ 0.13517121+0.j]
 [ 0.19684061+0.j]
 [-0.20605239+0.j]
 [ 0.00926426+0.j]
 [ 0.36173081+0.j]
 [ 0.16123148+0.j]
 [-0.08965996+0.j]].
Ga_m method for 45 gives 
[[ 0.0911163 ]
 [-0.12859611]
 [ 0.00287092]
 [-0.18480147]
 [ 0.25221397]
 [-0.09474533]
 [